# **Assignment 3  (From Scratch)**

## **Penalized Logistic Regression**

- **Programmers:**
  - Shaun Pritchard
  - Ismael A Lopez
- **Date:** 11-15-2021
- **Assignment:** 2
- **Prof:** M.DeGiorgio

<hr>

### **Overview: Assignment 3**

- In this assignment you will still be analyzing human genetic data from 𝑁 = 183 training
observations (individuals) sampled across the world. The goal is to fit a model that can predict
(classify) an individual’s ancestry from their genetic data that has been projected along 𝑝 = 10
top principal components (proportion of variance explained is 0.2416) that we use as features
rather than the raw genetic data

- Using ridge regression, fit a penalized (regularized) logistic (multinomial) regression with model parameters obtained by batch gradient descent. Based on K = 5 continental ancestries (African, European, East Asian, Oceanian, or Native American), predictions will be made. Ridge regression will permit parameter shrinkage (tuning parameter 𝜆 ≥ 0) to mitigate overfitting. In order to infer the bestfit model parameters on the training dataset, the tuning parameter will be selected using five-fold cross validation. After training, the model will be used to predict new test data points.



In [ ]:

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn

In [ ]:
train_df = pd.read_csv('TrainingData_N183_p10.csv')
test_df = pd.read_csv('TestData_N111_p10.csv')

In [ ]:
train_df.head(2)

In [ ]:
test_df.head(2)

In [ ]:
# recode the categories
class_names = train_df['Ancestry'].unique().tolist()
n_classes = len(class_names)
train_df['AncestryRecoded'] = train_df['Ancestry'].apply(lambda x: class_names.index(x))
train_df.head(2)

In [ ]:
train_df.shape

In [ ]:
# Seperate dependant categorical feature data
Y_train_names = train_df['Ancestry'].tolist()
Y_test_names = test_df['Ancestry'].tolist()

In [ ]:
# Separate training feature predictors from responses
X_train = np.float32(train_df.to_numpy()[:, :-2])
Y_train = train_df['AncestryRecoded'].to_numpy()

In [ ]:
# Separate test feature predictors from responses
X_test = np.float32(test_df.to_numpy()[:, :-1])

In [ ]:
X_train.shape

## **Set Global Vairibles**

In [ ]:
# Set local variables
# 9-Tuning Parms
# λ  =  10 ** np.arange(-4., 4.)
lambdas =  10 ** np.arange(-4., 4.)

# 6 learning & convergence rate
# α =  1e-4
alpha =  1e-4

# K-folds
nfolds = 5

# itterations
n_iters = 10000 

#log base of lambda
# λ_log = np.log10(λ) 

# Set verbose to True
verbose = True

# Set n x m matrix variable
preds = X_train

# Set n vector variable
responses  = Y_train

## **Instantiate Data**

In [ ]:
# encode Y response in CV
def _make_one_hot_responses(response_vector, n_classes):
   response_vector = np.int64(response_vector)
   n_samples = response_vector.shape[0]
   response_mat = np.zeros([n_samples, n_classes])
   response_mat[np.arange(n_samples), response_vector] = 1
   return response_mat

In [ ]:
def _shuffle_data(preds, responses):
    data = np.concatenate((preds, responses[:, None]), 1)
    np.random.shuffle(data)
    return data[:, :-1], data[:, -1]

In [ ]:
# shuffle the data
x, y = _shuffle_data(preds, responses)

In [ ]:
# get number of samples and number of features
n_samples = x.shape[0]
n_preds = x.shape[1]     

In [ ]:
# get number of training classes 
n_classes = np.unique(y).size

In [ ]:
 # make one-hot response mat
 y = _make_one_hot_responses(y, n_classes)      

In [ ]:
# matrix to store the cross validation results 
cv_vals = np.zeros([nfolds, len(lambdas)])

In [ ]:
# determine the number of validation samples and their inds based on nfolds 
n_val_samples = int(np.ceil(n_samples / nfolds)) 
val_inds = list(range(0, n_samples, n_val_samples))

In [ ]:
# create a tensor to store the trained coefficient vectors
B_trained = np.zeros([nfolds, len(lambdas), n_preds + 1, n_classes])

## **Implemnt functions**

In [ ]:
# Standardize X
def _standardize(x, mean_vec, std_vec):
   return (x - mean_vec) / std_vec 

In [ ]:
# def _initialize_B():
#     return np.zeros([n_preds + 1, n_classes])

In [ ]:
def predict(x):
    # assert(mean_vec is not None and std_vec is not None), \
    # 'Model must be trained before predicting.'
  x = _standardize(x, mean_vec, std_vec)
  x = _add_intercept(x)
  preds = np.exp(np.matmul(x, B))
  return preds / np.sum(preds, 1)[:, None]

In [ ]:
def _get_folds(val_ind):
    if val_ind + n_val_samples <= n_samples:
        val_inds = np.arange(val_ind, val_ind + n_val_samples)
    else:
        val_inds = np.arange(val_ind, n_samples)
            
    x_val = x[val_inds]
    x_train = np.delete(x, val_inds, axis = 0)
    y_val = y[val_inds]
    y_train = np.delete(y, val_inds, axis = 0)
    return x_train, x_val, y_train, y_val

In [ ]:
def score(x, y, B):
    unnorm_prob_mat = np.exp(np.matmul(x, B))
    norm_prob_mat = unnorm_prob_mat / np.sum(unnorm_prob_mat, 1)[:, None]
    ce = -(1 / x.shape[0]) * np.sum(np.sum(y * np.log10(norm_prob_mat), 1))
    return ce

In [ ]:
def _add_intercept(x):
    intercept_col = np.ones([x.shape[0], 1])
    return np.concatenate((intercept_col, x), 1)

In [ ]:
def LogisticRegresion(x, y, B, lambda_):

    unnorm_prob_mat = np.exp(np.dot(x, B))

    norm_prob_mat = unnorm_prob_mat / np.sum(unnorm_prob_mat, 1)[:, None]

    intercept_mat = B.copy()

    intercept_mat[1:] = 0
    
    B = B + alpha * (np.matmul(np.transpose(x), y - norm_prob_mat) - 2 * lambda_ * (B - intercept_mat))
    return B

## **Penlized Ridge CV**

In [ ]:
# Original function
for i_lambda, lambda_ in enumerate(lambdas):
    for i_fold, val_ind in zip(range(nfolds), val_inds):
        # get the folds
        x_train, x_val, y_train, y_val = _get_folds(val_ind)


        # standardize x 
        mean_vec, std_vec = np.mean(x_train, 0), np.std(x_train, 0)
        x_train = _standardize(x_train, mean_vec, std_vec)
        x_val = _standardize(x_val, mean_vec, std_vec)
                
        # add intercept column to design matrix
        x_train = _add_intercept(x_train)
        x_val = _add_intercept(x_val)


        # initialize Beta for this lambda and fold
        B =  np.zeros([n_preds + 1, n_classes])

        for iter in range(n_iters):
            B = LogisticRegresion(x_train, y_train, B, lambda_)

        # score this model and store the value 
        cv_vals[i_fold, i_lambda] = score(x_val, y_val, B)
                
        # save this coefficient vector
        B_trained[i_fold, i_lambda] = B

    # # find the best lambda and retrain model
    # best_lambda = lambdas[np.argmin(np.mean(cv_vals, 0))]
    # mean_vec, std_vec = np.mean(x, 0), np.std(x, 0)
    # x = _standardize(x, mean_vec, std_vec)
    # x = _add_intercept(x)
    # y = y.copy()
    # B = np.zeros([n_preds + 1, n_classes])
        
    # for iter in range(n_iters):
    #     B = LogisticRegresion(x, y, B, best_lambda)

In [ ]:
mean_betas = np.mean(logistic_ridge.B_trained, 0)

for class_ind, class_name in enumerate(class_names):
    mean_beta_k = mean_betas[..., class_ind]

    for pred_num in range(1, 1 + logistic_ridge.n_preds):
        plt.plot(
            logistic_ridge.lambdas, 
            mean_beta_k[:, pred_num],
            label = 'PC{}'.format(pred_num)
        )
    
    plt.xscale('log')
    plt.legend(bbox_to_anchor = (1.05, 1), loc = 'upper left')
    plt.xlabel('Log Lambda')
    plt.ylabel('Coefficient Value')
    plt.title(class_name)
    plt.show()

In [ ]:
se = np.std(logistic_ridge.cv_vals, 0) / np.sqrt(logistic_ridge.cv_vals.shape[0])
plt.errorbar(
    logistic_ridge.lambdas, 
    np.mean(logistic_ridge.cv_vals, 0),
    yerr = se
)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Log Lambda')
plt.ylabel('Log CV MSE +/- 1 SE')
plt.show()

In [ ]:
print('Optimal lambda value: {}'.format(logistic_ridge.best_lambda))

In [ ]:
Y_train_hat = logistic_ridge.predict(X_train)
Y_train_class = np.argmax(Y_train_hat, 1)
print('Training Accuracy: {}'.format(np.mean(Y_train_class == Y_train)))

In [ ]:

Y_test_hat = logistic_ridge.predict(X_test)
Y_test_class = np.argmax(Y_test_hat, 1)

In [ ]:

# add these predictions to the test dataframe
new_df_col_names = ['{}Prob'.format(class_name) for class_name in class_names] + ['ClassPredInd']
prob_and_ind_array = np.concatenate((Y_test_hat, Y_test_class[:, None]), 1)
new_df = pd.DataFrame(prob_and_ind_array, columns = new_df_col_names)
test_anc_with_preds = pd.concat([test_df['Ancestry'], new_df], axis = 1)
test_anc_with_preds['ClassPredName'] = test_anc_with_preds['ClassPredInd'].apply(lambda x: class_names[int(x)])

In [ ]:
print(test_anc_with_preds.to_string())

In [ ]:
test_true_with_probs = test_anc_with_preds.loc[:, 'Ancestry':'NativeAmericanProb']
test_true_with_probs_long = pd.melt(
    test_true_with_probs,
    id_vars = ['Ancestry'],
    var_name = 'AncestryPred',
    value_name = 'Probability'
)
test_true_with_probs_long['AncestryPred'] = test_true_with_probs_long['AncestryPred'].apply(lambda x: x.split('Prob')[0])

In [ ]:
seaborn.catplot(
    data = test_true_with_probs_long[test_true_with_probs_long['Ancestry'] != 'Unknown'],
    kind = 'bar',
    x = 'Ancestry',
    y = 'Probability',
    hue = 'AncestryPred',
    ci = "sd"
)
plt.show()